In [1]:
import numpy
nrOfListEntries = 10**6
a = numpy.random.normal(3, 1, size = nrOfListEntries).astype(numpy.float32)

In [2]:
%load_ext Cython


In [3]:


%%cython

"""
Implementation of mean and standard deviation calculation using
cython
"""
#from __future__ import with_statement
from cython.parallel import prange
import numpy
cimport numpy
cimport cython
from libc.math cimport sqrt

ctypedef numpy.float32_t dtype_t

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef numpy.ndarray[dtype_t, ndim=1] cython_stat_parallel(
        numpy.ndarray[dtype_t, ndim=1] l):
    """
    calculate mean and standard deviation of data stored in a list
    using cython parallel.
    Args:
        l numpy array containing numbers
    Returns:
        list [mean, standardDeviation] 
    """
    cdef double average, standard_deviation, skewness, kurtosis, tmp
    cdef double accumulator = 0.0
    cdef long N, i

    N = len(l)
    for i in range(N, nogil = True):
        accumulator += l[i]
    average = accumulator / N
    accumulator = 0.0
    for i in range(N, nogil = True):
        tmp = l[i] - average
        accumulator += tmp * tmp
        skewness += tmp * tmp * tmp
        kurtosis += tmp * tmp * tmp * tmp
    standard_deviation = sqrt(accumulator / (N - 1))
    skewness = skewness / pow(standard_deviation, 3)
    kurtosis = kurtosis / pow(standard_deviation, 4)

    result = numpy.array((average, standard_deviation, skewness, kurtosis), numpy.float32)
    return result

%timeit cython_stat(a)
print(cython_stat(a))

SyntaxError: invalid syntax (<ipython-input-3-16688b390ca3>, line 11)